In [3]:
#!/usr/bin/env python3
"""
vLLM Tensor Parallelism Test - DeepSeek-V2-Lite-Coder
Tests bit-exact reproducibility across multiple runs with TP
Automatically finds txt/pdf files in current directory
Uses standard OpenAI-compatible message format
"""

# ============================================================================
# SUPPRESS VERBOSE LOGGING
# ============================================================================
import os
os.environ['HF_HOME'] = '/tmp/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/tmp/hf_cache'

# Suppress vLLM verbose output
os.environ['VLLM_LOGGING_LEVEL'] = 'WARNING'
os.environ['VLLM_CONFIGURE_LOGGING'] = '0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import warnings
warnings.filterwarnings('ignore')

import logging
# Suppress vLLM and related libraries
logging.getLogger('vllm').setLevel(logging.ERROR)
logging.getLogger('vllm.engine').setLevel(logging.ERROR)
logging.getLogger('vllm.worker').setLevel(logging.ERROR)
logging.getLogger('vllm.executor').setLevel(logging.ERROR)
logging.getLogger('transformers').setLevel(logging.ERROR)
logging.getLogger('torch').setLevel(logging.ERROR)

# Keep download progress - only allow INFO for huggingface_hub downloads
logging.getLogger('huggingface_hub').setLevel(logging.INFO)
logging.getLogger('huggingface_hub.file_download').setLevel(logging.INFO)

# ============================================================================
# IMPORTS
# ============================================================================


os.environ['HF_HOME'] = '/tmp/hf_cache'
os.environ['TRANSFORMERS_CACHE'] = '/tmp/hf_cache'

from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import numpy as np
from datetime import datetime
import json
import torch
import glob

# ============================================================================
# CONFIGURATION
# ============================================================================

# Model configuration
MODEL_NAME = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
TENSOR_PARALLEL_SIZE = 1
MAX_MODEL_LEN = 128000
GPU_MEMORY_UTILIZATION = 0.9

# Generation configuration
MAX_TOKENS = 40
NUM_REPETITIONS = 20
TEMPERATURE = 0.0  # Greedy decoding
SEED = 42
TOP_LOGPROBS = 10

# Prompt source - finds first txt or pdf in current directory
AUTO_FIND_FILE = True  # Set to False to use hardcoded content

# User task
USER_TASK = "Please provide a detailed summary of the following text."

# Hardcoded content (used if AUTO_FIND_FILE=False or no files found)
HARDCODED_CONTENT = """The development of large language models has fundamentally transformed natural language processing and artificial intelligence more broadly. These models, trained on vast corpora of text data, have demonstrated remarkable capabilities across a wide range of tasks, from translation and summarization to question answering and creative writing."""

# ============================================================================
# FILE LOADING UTILITIES
# ============================================================================

def find_prompt_file():
    """Find first txt or pdf file in current directory"""
    cwd = os.getcwd()
    
    # Look for txt files first, then pdf
    txt_files = glob.glob(os.path.join(cwd, "*.txt"))
    pdf_files = glob.glob(os.path.join(cwd, "*.pdf"))
    
    if txt_files:
        return txt_files[0]
    elif pdf_files:
        return pdf_files[0]
    else:
        return None

def load_text_from_file(filepath):
    """Load text from txt or pdf file"""
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File not found: {filepath}")
    
    if filepath.endswith('.txt'):
        with open(filepath, 'r', encoding='utf-8') as f:
            text = f.read()
        print(f"Loaded {len(text)} characters from txt file")
        return text
    
    elif filepath.endswith('.pdf'):
        try:
            import PyPDF2
        except ImportError:
            raise ImportError("PyPDF2 required for PDF loading. Install with: pip install PyPDF2")
        
        text = []
        with open(filepath, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            num_pages = len(pdf_reader.pages)
            print(f"Loading {num_pages} pages from PDF...")
            
            for page_num, page in enumerate(pdf_reader.pages, 1):
                page_text = page.extract_text()
                text.append(page_text)
                if page_num % 10 == 0:
                    print(f"  Processed {page_num}/{num_pages} pages")
        
        full_text = '\n'.join(text)
        print(f"Loaded {len(full_text)} characters from PDF ({num_pages} pages)")
        return full_text
    
    else:
        raise ValueError(f"Unsupported file type: {filepath}. Use .txt or .pdf")

# ============================================================================
# PROMPT LOADING
# ============================================================================

print("=" * 80)
print("vLLM LOGPROBS EXTRACTION TEST")
print("=" * 80)
print()

# Load document content
prompt_file = None
if AUTO_FIND_FILE:
    prompt_file = find_prompt_file()
    if prompt_file:
        print(f"Found file: {os.path.basename(prompt_file)}")
        DOCUMENT_CONTENT = load_text_from_file(prompt_file)
        print()
    else:
        print("No txt/pdf files found in current directory")
        print("Using hardcoded content")
        DOCUMENT_CONTENT = HARDCODED_CONTENT
        print()
else:
    DOCUMENT_CONTENT = HARDCODED_CONTENT
    print("Using hardcoded content")
    print()

# Prepare messages in standard OpenAI format
# vLLM's tokenizer will apply the model's chat template to convert this to a string
messages = [
    {"role": "user", "content": f"{USER_TASK}\n\n{DOCUMENT_CONTENT}"}
]

print("Using standard OpenAI-compatible message format")
print(f"Message content length: {len(messages[0]['content'])} characters")
print("Note: Chat template will be applied to convert messages to string prompt")
print()

# ============================================================================
# TOKENIZER PRECHECK
# ============================================================================

print("Loading tokenizer to validate prompt length...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    cache_dir='/tmp/hf_cache',
    trust_remote_code=True
)

# Apply chat template to convert messages to string prompt
# This is the actual prompt that will be used for generation
prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("Chat template applied successfully")
print()

# Tokenize prompt
prompt_tokens = tokenizer.encode(prompt_text)
prompt_length = len(prompt_tokens)

print(f"Prompt statistics:")
print(f"  Characters: {len(prompt_text):,}")
print(f"  Tokens: {prompt_length:,}")
print(f"  Max model length: {MAX_MODEL_LEN:,}")
print(f"  Generation tokens: {MAX_TOKENS}")
print(f"  Total required: {prompt_length + MAX_TOKENS:,}")
print()

# Validate length
if prompt_length > MAX_MODEL_LEN:
    print(f"❌ ERROR: Prompt is too long!")
    print(f"  Prompt has {prompt_length:,} tokens")
    print(f"  Model max is {MAX_MODEL_LEN:,} tokens")
    print(f"  Exceeds by {prompt_length - MAX_MODEL_LEN:,} tokens")
    print()
    print("Solutions:")
    print(f"  1. Increase MAX_MODEL_LEN to at least {prompt_length + MAX_TOKENS}")
    print(f"  2. Truncate/reduce the prompt")
    exit(1)

if prompt_length + MAX_TOKENS > MAX_MODEL_LEN:
    print(f"⚠️  WARNING: Prompt + generation may exceed context")
    print(f"  Prompt: {prompt_length:,} tokens")
    print(f"  Generation: {MAX_TOKENS} tokens")
    print(f"  Total: {prompt_length + MAX_TOKENS:,} tokens")
    print(f"  Model max: {MAX_MODEL_LEN:,} tokens")
    print(f"  Consider increasing MAX_MODEL_LEN to {prompt_length + MAX_TOKENS + 100}")
    print()
else:
    print(f"✓ Prompt length validation passed")
    print(f"  Remaining capacity: {MAX_MODEL_LEN - prompt_length - MAX_TOKENS:,} tokens")
    print()

# ============================================================================
# MODEL LOADING
# ============================================================================

print("Configuration:")
print(f"  Model: {MODEL_NAME}")
print(f"  Tensor parallel: {TENSOR_PARALLEL_SIZE}")
print(f"  Max model len: {MAX_MODEL_LEN:,}")
print(f"  Max tokens: {MAX_TOKENS}")
print(f"  Temperature: {TEMPERATURE}")
print(f"  Seed: {SEED}")
print(f"  Repetitions: {NUM_REPETITIONS}")
print()

print("Loading model...")
print("Note: vLLM will automatically detect FP8 quantization config")
print()

llm = LLM(
    model=MODEL_NAME,
    tensor_parallel_size=TENSOR_PARALLEL_SIZE,
    max_model_len=MAX_MODEL_LEN,
    gpu_memory_utilization=GPU_MEMORY_UTILIZATION,
    trust_remote_code=True,
    seed=SEED,
    enforce_eager=True,  # Disable cudagraph for determinism
    enable_prefix_caching=False  # Disable prefix caching for clean experiment
)

print("Model loaded successfully!")
print()

# ============================================================================
# SAMPLING CONFIGURATION
# ============================================================================

sampling_params = SamplingParams(
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    seed=SEED,
    logprobs=TOP_LOGPROBS,
    skip_special_tokens=False
)

print("Sampling parameters:")
print(f"  Temperature: {TEMPERATURE}")
print(f"  Max tokens: {MAX_TOKENS}")
print(f"  Seed: {SEED}")
print(f"  Top logprobs: {TOP_LOGPROBS}")
print()

# ============================================================================
# WARMUP
# ============================================================================

print("Running warmup...")
# vLLM's LLM.generate() expects string prompts, not message dicts
# The chat template was already applied above during validation
warmup_output = llm.generate(prompt_text, sampling_params=sampling_params)
print(f"Warmup complete - generated {len(warmup_output[0].outputs[0].token_ids)} tokens")
print()

# ============================================================================
# MAIN EXPERIMENT
# ============================================================================

print("=" * 80)
print("RUNNING EXPERIMENT")
print("=" * 80)
print()

results_tokens = []
results_logprobs = []
results_texts = []
results_distributions = []

for rep in range(NUM_REPETITIONS):
    print(f"Repetition {rep + 1}/{NUM_REPETITIONS}...")
    
    # Use the string prompt (chat template already applied)
    outputs = llm.generate(prompt_text, sampling_params=sampling_params)
    output = outputs[0]
    
    # Extract token IDs
    token_ids = output.outputs[0].token_ids
    results_tokens.append(token_ids)
    
    # Extract generated text
    text = output.outputs[0].text
    results_texts.append(text)
    
    # Extract logprobs for selected tokens
    logprobs_data = output.outputs[0].logprobs
    selected_logprobs = [lp[token_ids[i]].logprob for i, lp in enumerate(logprobs_data)]
    results_logprobs.append(np.array(selected_logprobs))
    
    # Extract full top-k distributions
    rep_distributions = []
    for position_logprobs in logprobs_data:
        # Get top-k sorted by logprob (descending)
        sorted_items = sorted(position_logprobs.items(), 
                            key=lambda x: x[1].logprob, 
                            reverse=True)[:TOP_LOGPROBS]
        rep_distributions.append([(tok, lp.logprob) for tok, lp in sorted_items])
    results_distributions.append(rep_distributions)
    
    print(f"  Generated {len(token_ids)} tokens")

print()
print("All repetitions complete!")
print()

# ============================================================================
# ANALYSIS
# ============================================================================

print("=" * 80)
print("ANALYSIS")
print("=" * 80)
print()

# Check token sequence identity
print("Checking token sequences...")
tokens_identical = all(
    results_tokens[0] == results_tokens[i] 
    for i in range(1, NUM_REPETITIONS)
)
print(f"Token sequences identical: {tokens_identical}")

if not tokens_identical:
    print("\n⚠️  Token sequences differ!")
    for i in range(1, NUM_REPETITIONS):
        if results_tokens[0] != results_tokens[i]:
            diff_positions = [
                j for j in range(min(len(results_tokens[0]), len(results_tokens[i])))
                if results_tokens[0][j] != results_tokens[i][j]
            ]
            print(f"  Rep 0 vs Rep {i}: {len(diff_positions)} positions differ")
            if diff_positions:
                print(f"    First difference at position {diff_positions[0]}")

# Check logprobs for selected tokens
print("\nChecking selected token logprobs...")
first_logprobs = results_logprobs[0]
logprobs_exact = all(
    np.allclose(first_logprobs, results_logprobs[i], rtol=0, atol=1e-10)
    for i in range(1, NUM_REPETITIONS)
)
print(f"Selected token logprobs bit-exact: {logprobs_exact}")

# Check top-k distributions
print("\nChecking full top-k distributions...")
distributions_exact = True
distribution_mismatches = []

first_dist = results_distributions[0]
for rep_idx in range(1, NUM_REPETITIONS):
    for pos_idx in range(len(first_dist)):
        dist_a = first_dist[pos_idx]
        dist_b = results_distributions[rep_idx][pos_idx]
        
        # Check if token IDs match in same order
        tokens_match = [t[0] for t in dist_a] == [t[0] for t in dist_b]
        
        # Check if logprobs are bit-exact
        if tokens_match:
            logprobs_match = all(
                abs(dist_a[i][1] - dist_b[i][1]) < 1e-10 
                for i in range(len(dist_a))
            )
            if not logprobs_match:
                distributions_exact = False
                distribution_mismatches.append((rep_idx, pos_idx))
        else:
            distributions_exact = False
            distribution_mismatches.append((rep_idx, pos_idx))

print(f"Top-k distributions bit-exact: {distributions_exact}")

if not distributions_exact:
    print(f"\n⚠️  Found {len(distribution_mismatches)} position mismatches in distributions")
    if len(distribution_mismatches) <= 5:
        for rep_idx, pos_idx in distribution_mismatches:
            print(f"  Rep 0 vs Rep {rep_idx}, position {pos_idx}")
    else:
        print(f"  First 5: {distribution_mismatches[:5]}")

if not logprobs_exact:
    print("\nL2 distances:")
    l2_distances = []
    for i in range(1, NUM_REPETITIONS):
        l2 = np.linalg.norm(first_logprobs - results_logprobs[i])
        l2_distances.append(l2)
        print(f"  Rep 0 vs Rep {i}: L2 = {l2:.6e}")
    
    print(f"\nMax L2: {max(l2_distances):.6e}")
    print(f"Mean L2: {np.mean(l2_distances):.6e}")
    
    # Element-wise statistics
    all_logprobs = np.array(results_logprobs)
    std_per_token = all_logprobs.std(axis=0)
    print(f"\nPer-token std statistics:")
    print(f"  Mean: {std_per_token.mean():.6e}")
    print(f"  Max: {std_per_token.max():.6e}")
    print(f"  Median: {np.median(std_per_token):.6e}")

print()

# ============================================================================
# VERDICT
# ============================================================================

print("=" * 80)
print("VERDICT")
print("=" * 80)
print()

if tokens_identical and logprobs_exact and distributions_exact:
    print("✓ PERFECT REPRODUCIBILITY")
    print("  - Token sequences: bit-exact")
    print("  - Selected token logprobs: bit-exact")
    print("  - Full top-k distributions: bit-exact")
    print("  - vLLM + ds_v2lite_instruct is deterministic for this config")
elif tokens_identical and logprobs_exact and not distributions_exact:
    print("⚠️  SELECTED TOKENS EXACT, DISTRIBUTIONS VARY")
    print("  - Token sequences: bit-exact")
    print("  - Selected token logprobs: bit-exact")
    print("  - Top-k distributions: numerical variation detected")
    print("  → May indicate computational instability in non-selected paths")
elif tokens_identical and not logprobs_exact:
    print("⚠️  TOKENS IDENTICAL, LOGPROBS VARY")
    print("  - Token sequences: bit-exact")
    print("  - Logprobs: small numerical variation")
    max_l2 = max(l2_distances) if not logprobs_exact else 0.0
    if max_l2 < 1e-6:
        print(f"  - Variation very small (L2={max_l2:.2e})")
        print("  → Likely acceptable for forensics")
    else:
        print(f"  - Variation notable (L2={max_l2:.2e})")
        print("  → Investigate noise source")
else:
    print("❌ TOKEN SEQUENCES DIFFER")
    print("  - This should NOT happen with temperature=0")
    print("  → Something is wrong, investigate")

print()

# ============================================================================
# SAVE RESULTS
# ============================================================================

output_data = {
    "experiment": "ds_v2lite_instruct_logprobs_test",
    "timestamp": datetime.now().isoformat(),
    "prompt_source": "file" if prompt_file else "hardcoded",
    "prompt_file": os.path.basename(prompt_file) if prompt_file else None,
    "prompt_text": prompt_text,  # Save actual prompt for reproducibility
    "prompt_length_chars": len(prompt_text),
    "prompt_length_tokens": prompt_length,
    "message_format": "openai_compatible",
    "config": {
        "model": MODEL_NAME,
        "tensor_parallel": TENSOR_PARALLEL_SIZE,
        "max_model_len": MAX_MODEL_LEN,
        "max_tokens": MAX_TOKENS,
        "repetitions": NUM_REPETITIONS,
        "temperature": TEMPERATURE,
        "seed": SEED,
        "warmup_enabled": True,
        "prefix_caching_disabled": True,
        "enforce_eager": True,
        "top_logprobs": TOP_LOGPROBS
    },
    "results": {
        "tokens_identical": tokens_identical,
        "logprobs_exact": logprobs_exact,
        "distributions_exact": distributions_exact,
        "perfect_reproducibility": tokens_identical and logprobs_exact and distributions_exact
    },
    "token_sequences": results_tokens,
    "logprobs_vectors": [lp.tolist() for lp in results_logprobs],
    "generated_texts": results_texts,
    "top_k_distributions": [
        [[(int(tok), float(prob)) for tok, prob in dist] for dist in rep_dists]
        for rep_dists in results_distributions
    ]
}

output_file = f"vllm_ds_v2instruct_test_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_file, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"Results saved to: {output_file}")
print()
print("=" * 80)
print("TEST COMPLETE")
print("=" * 80)
print()

vLLM LOGPROBS EXTRACTION TEST

Found file: Verification-for-International-AI-Governance.pdf
Loading 172 pages from PDF...
  Processed 10/172 pages
  Processed 20/172 pages
  Processed 30/172 pages
  Processed 40/172 pages
  Processed 50/172 pages
  Processed 60/172 pages
  Processed 70/172 pages
  Processed 80/172 pages
  Processed 90/172 pages
  Processed 100/172 pages
  Processed 110/172 pages
  Processed 120/172 pages
  Processed 130/172 pages
  Processed 140/172 pages
  Processed 150/172 pages
  Processed 160/172 pages
  Processed 170/172 pages
Loaded 535619 characters from PDF (172 pages)

Using standard OpenAI-compatible message format
Message content length: 535677 characters
Note: Chat template will be applied to convert messages to string prompt

Loading tokenizer to validate prompt length...
Chat template applied successfully

Prompt statistics:
  Characters: 535,716
  Tokens: 124,366
  Max model length: 128,000
  Generation tokens: 40
  Total required: 124,406

✓ Prompt leng

(EngineCore_DP0 pid=3191) 2025-11-10 14:57:15,612 - INFO - autotuner.py:256 - flashinfer.jit: [Autotuner]: Autotuning process starts ...
(EngineCore_DP0 pid=3191) 2025-11-10 14:57:15,989 - INFO - autotuner.py:262 - flashinfer.jit: [Autotuner]: Autotuning process ends


Model loaded successfully!

Sampling parameters:
  Temperature: 0.0
  Max tokens: 40
  Seed: 42
  Top logprobs: 10

Running warmup...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  Generated 40 tokens
Repetition 20/20...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

  Generated 40 tokens

All repetitions complete!

ANALYSIS

Checking token sequences...
Token sequences identical: True

Checking selected token logprobs...
Selected token logprobs bit-exact: True

Checking full top-k distributions...
Top-k distributions bit-exact: True

VERDICT

✓ PERFECT REPRODUCIBILITY
  - Token sequences: bit-exact
  - Selected token logprobs: bit-exact
  - Full top-k distributions: bit-exact
  - vLLM + ds_v2lite_instruct is deterministic for this config

Results saved to: vllm_ds_v2instruct_test_20251110_150107.json

TEST COMPLETE

